# TRANSFORMER con PyTorch   

El objetivo de este notebook es proporcionar una comprensión completa de cómo construir un modelo Transformer utilizando PyTorch.    
El Transformer es uno de los modelos más potentes en el aprendizaje automático moderno. Han revolucionado el campo, particularmente en tareas de Procesamiento del Lenguaje Natural (PLN) como la traducción de idiomas y el resumen de textos.    

Las redes LSTM han sido sustituidas por Transformers en estas tareas debido a su capacidad para manejar dependencias de largo alcance y cálculos paralelos.

La herramienta utilizada en este notebook para construir el Transformer es `PyTorch`. PyTorch se ha convertido en un recurso para investigadores y desarrolladores en el ámbito del aprendizaje automático y la inteligencia artificial.

## Configuración de PyTorch
Antes de comenzar con la construcción de un Transformer, es esencial configurar correctamente el entorno de trabajo. 
Lo primero es instalar PyTorch.    

`PyTorch (versión estable actual - 2.6.0)`:

    `pip3 install torch torchvision torchaudio`

## Construir el modelo Transformer con PyTorch
Para construir el modelo Transformer son necesarios los siguientes pasos:

1. Importar las librerías y módulos
2. Definir los bloques de construcción básicos - Atención Multicabezal, Redes Feed-Forward de Posición, Codificación Posicional
3. Construcción del bloque codificador
4. Construcción del bloque decodificador
5. Combinación de las capas de codificación y descodificación para crear una red de transformadores completa.

### 1. Importar las librerías y módulos necesarios   

Primero se importa la librería PyTorch para disponer de la funcionalidad básica, el módulo de redes neuronales para crear redes neuronales, el módulo de optimización para entrenar redes y las funciones de utilidad de datos para manejar datos.    
Además, se importa el módulo matemático estándar de Python para operaciones matemáticas y el módulo copy para crear copias de objetos complejos.

Estas herramientas preparan las bases para definir la arquitectura del modelo, gestionar los datos y establecer el proceso de entrenamiento.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy

### 2. Definición de los componentes básicos: Multi-Head Attention, redes Feed-Forward posicionales, codificación posicional
#### Multi-Head Attention
El mecanismo de atención multicabezal calcula la atención entre cada par de posiciones de una secuencia. Consta de varias "cabezas de atención" que captan distintos aspectos de la secuencia de entrada.    

<img src="./img/multiples-bloques-atencionales-red-transformer.jpeg" width=300 heigth=100>   

In [2]:
# Definimos la clase para la capa de atención
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        # Garantizar que la dimensión del modelo (d_model) es divisible por el número de cabezas
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        # Inicializamos las dimensiones
        self.d_model = d_model # Dimension del modelo
        self.num_heads = num_heads # Numero de cabezas de atención
        self.d_k = d_model // num_heads # Dimensión de la clave(key), la consulta(query) y el valor(value) de cada cabeza
        
        # Capas lineales para transformar las entradas
        self.W_q = nn.Linear(d_model, d_model) # Transformación de consultas(query)
        self.W_k = nn.Linear(d_model, d_model) # Transformación clave(key)
        self.W_v = nn.Linear(d_model, d_model) # Transformación del valor(value)
        self.W_o = nn.Linear(d_model, d_model) # Transformación de la salida
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        # Calcular las puntuaciones de atención
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        # Aplicar máscara si está prevista (útil para evitar que se preste atención a determinadas partes como el relleno).
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        
        # Se aplica Softmax para obtener probabilidades de atención
        attn_probs = torch.softmax(attn_scores, dim=-1)
        
        # Multiplicar por los valores(values) para obtener el resultado final
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        # Reformateo de la entrada para tener num_heads para atención multicabezal
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        # Combinar las cabezas múltiples para volver de nuevo a la forma original
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        # Aplicar transformaciones lineales y separar las cabezas
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        # Ejecutar el scaled dot-product attention (producto escalar de atención), para obtener las salidas de atención
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        
        # Combinar cabezas y aplicar transformación de salida
        output = self.W_o(self.combine_heads(attn_output))
        return output

##### <u>*Definición e inicialización de clases:*</u>

````
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
````
La clase se define como una subclase de `nn.Module` de PyTorch.

- `d_model`: Dimensionalidad de la entrada.
- `num_heads`: El número de cabezas de atención en el que se va a dividir la entrada.   

La inicialización comprueba si `d_model` es divisible por num_heads y, a continuación, define los pesos de transformación para la consulta, la clave, el valor y la salida.   

##### <u>*Scaled Dot-Product Attention:*</u>   

`````
def scaled_dot_product_attention(self, Q, K, V, mask=None):
`````   

1. Cálculo de las puntuaciones de atención: `attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)`. Aquí, las puntuaciones de atención se calculan tomando el producto punto de consultas (Q) y claves (K), y luego escalando por la raíz cuadrada de la dimensión `clave (d_k)`.
2. Aplicación de la máscara: si se proporciona una máscara, se aplica a las puntuaciones de atención para enmascarar valores específicos.
3. Cálculo de la ponderación de la atención: Las puntuaciones de atención se pasan por una función softmax para convertirlas en probabilidades que sumen 1.
4. Cálculo del resultado: La salida final de la atención se calcula multiplicando los pesos de atención por los valores (V).   

##### <u>*Splitting Heads (División de cabezales):*</u>   

`````
def split_heads(self, x):
`````

Este método reformatea la entrada `x` usando el formato: `(batch_size, num_heads, seq_length, d_k)`. Permite al modelo procesar múltiples cabezas de atención simultáneamente, permitiendo el cálculo paralelo.


##### <u>*Método de 'forwarding'(propagación hacia adelante):*</u>   


El método de avance es donde se realiza el cálculo real:

1. *Aplicar transformaciones lineales*: Las consultas (Q), claves (K) y valores (V) se pasan primero por transformaciones lineales utilizando los pesos definidos en la inicialización.
2. *Dividir cabezas*: Las transformadas Q, K, V se dividen en múltiples cabezas utilizando el método `split_heads`.
3. *Aplicar el producto escalar*: Se llama al método `scaled_dot_product_attention` en las cabezas separadas.
4. *Combinar cabezas*: Los resultados de cada cabeza se combinan de nuevo en un único tensor utilizando el método `combine_heads`.
5. *Aplicar transformación de salida*: Finalmente, el tensor combinado se pasa a través de una transformación lineal de salida.   


En resumen, la clase `MultiHeadAttention` encapsula el mecanismo de atención multicabezal comúnmente utilizado en los modelos transformadores.    
Se encarga de dividir la entrada en múltiples cabezas de atención, aplicando atención a cada cabeza, y luego combinando los resultados.    
De este modo, el modelo puede capturar varias relaciones en los datos de entrada a diferentes escalas, mejorando la capacidad expresiva del modelo.

#### Redes Feed-Forward posicionales

In [4]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

##### <u>*Definición de clase:*</u>   

La clase es una subclase de `nn.Module` de PyTorch, heredando todas las funcionalidades necesarias para trabajar con capas de redes neuronales.

`````
class PositionWiseFeedForward(nn.Module):
`````   

***Inicialización:***
````
def __init__(self, d_model, d_ff):
    super(PositionWiseFeedForward, self).__init__()
    self.fc1 = nn.Linear(d_model, d_ff)
    self.fc2 = nn.Linear(d_ff, d_model)
    self.relu = nn.ReLU()   
```` 
1. `d_model`: Dimensionalidad de la entrada y salida del modelo.
2. `d_ff`: Dimensionalidad de la capa interna de la red feed-forward.
3. `self.fc1` y `self.fc2`: Dos capas totalmente conectadas (lineales) con dimensiones de entrada y salida definidas por `d_model` y `d_ff`.
4. `self.relu`: Función de activación ReLU (Rectified Linear Unit), que introduce no linealidad entre las dos capas lineales.   

***Método forward***   

`````
def forward(self, x):
    return self.fc2(self.relu(self.fc1(x)))
`````

1. `x`: La entrada a la red feed-forward.
2. `self.fc1(x)`: La entrada pasa primero por la primera capa lineal (fc1).
3. `self.relu(...)`: La salida de fc1 se pasa a través de una función de activación ReLU. ReLU sustituye todos los valores negativos por ceros, introduciendo no linealidad en el modelo.
4. `self.fc2(...)`: La salida activada se pasa a través de la segunda capa lineal (fc2), produciendo la salida final.   

##### <u>*Resumen*</u>
La clase `PositionWiseFeedForward` define una red neuronal feed-forward que consiste en dos capas lineales con una función de activación ReLU entre ellas. En el contexto de los modelos de transformador, esta red de avance se aplica a cada posición por separado y de forma idéntica.    

Por tanto, esto ayuda a transformar las características aprendidas por los mecanismos de atención dentro del transformador, actuando como un paso de procesamiento adicional para los resultados de la atención.

#### Positional Encoding   

La codificación posicional se utiliza para "inyectar" la información de la posición de cada token en la secuencia de entrada.    
Utiliza las funciones seno y coseno de distintas frecuencias para generar la codificación posicional.

In [5]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

##### <u>*Definición de la clase:*</u>   

`````
class PositionalEncoding(nn.Module):
`````
    
La clase se define como una subclase de `nn.Module` de PyTorch, usándola como una capa estándar de PyTorch.   

***Inicialización:***

`````
def __init__(self, d_model, max_seq_length):
    super(PositionalEncoding, self).__init__()
    
    pe = torch.zeros(max_seq_length, d_model)
    position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
    
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    
    self.register_buffer('pe', pe.unsqueeze(0))
`````    
1. `d_model`: La dimensión de la entrada del modelo.
2. `max_seq_length`: La longitud máxima de la secuencia para la que se precalculan las codificaciones posicionales.
3. `pe`: Un tensor relleno de ceros, que se poblará con codificaciones posicionales.
4. `position`: Un tensor que contiene los índices de posición para cada posición en la secuencia.
5. `div_term`: Un término utilizado para escalar los índices de posición de una manera específica.
6. La función seno se aplica a los índices pares y la función coseno a los impares de `pe`.
7. Por último, pe se registra como un buffer, lo que significa que formará parte del estado del módulo pero no se considerará un parámetro entrenable.    

***Método Forward:***

`````
def forward(self, x):
    return x + self.pe[:, :x.size(1)]
`````   
El método forward simplemente añade las codificaciones posicionales a la entrada `x`.

Utiliza los primeros elementos `x.size(1)` de `pe` para garantizar que las codificaciones posicionales coincidan con la longitud de secuencia real de `x`.   


##### <u>Resumen</u>

La clase `PositionalEncoding` añade información sobre la posición de los tokens dentro de la secuencia. Dado que el modelo de transformer carece de conocimiento inherente del orden de las fichas (debido a su mecanismo de autoatención), esta clase ayuda al modelo a considerar la posición de las fichas en la secuencia.    

Las funciones sinusoidales utilizadas se eligen para permitir que el modelo aprenda fácilmente a prestar atención a las posiciones relativas, ya que producen una codificación única y suave para cada posición en la secuencia.

### 3. Bloques de codificación   

<img src="./img/codificador-1-red-transformer.jpeg" width=300>    


In [6]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

***Definición de la clase:***   
`````
class EncoderLayer(nn.Module):
`````   

La clase se define como una subclase de `nn.Module` de PyTorch y se utiliza como un bloque de redes neuronales en PyTorch.   

***Inicialización:***   

````
def __init__(self, d_model, num_heads, d_ff, dropout):
    super(EncoderLayer, self).__init__()
    self.self_attn = MultiHeadAttention(d_model, num_heads)
    self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
    self.norm1 = nn.LayerNorm(d_model)
    self.norm2 = nn.LayerNorm(d_model)
    self.dropout = nn.Dropout(dropout)
````   

Parámetros:

1. `d_model`: La dimensionalidad de la entrada.
2. `num_heads`: El número de cabezas de atención en la atención multicabeza.
3. `d_ff`: La dimensionalidad de la capa interna en la red feed-forward en función de la posición.
4. `dropout`: La *tasa de abandono* utilizada para la regularización.   

Componentes:

1. `self.self_attn`: Mecanismo de atención multicabezal.
2. `self.feed_forward`: Red neuronal de avance en función de la posición.
3. `self.norm1` y `self.norm2`: Normalización de la capa, aplicada para suavizar la entrada de la capa.
4. `self.dropout`: Capa de *dropout*, utilizada para evitar el sobreajuste estableciendo aleatoriamente algunas activaciones a cero durante el entrenamiento.

***Método forward:***   

````
def forward(self, x, mask):
    attn_output = self.self_attn(x, x, x, mask)
    x = self.norm1(x + self.dropout(attn_output))
    ff_output = self.feed_forward(x)
    x = self.norm2(x + self.dropout(ff_output))
    return x
````    

*Entrada:*

`x`: La entrada de la capa codificadora.
`mask`: Máscara opcional para ignorar ciertas partes de la entrada.

*Pasos del proceso:*

1. **Autoatención**: La entrada `x` pasa por el mecanismo de autoatención multicabezal.
2. **Añadir y normalizar (después de la atención)**: El resultado de la atención se añade a la entrada original (conexión residual), seguido de la eliminación y normalización mediante `norm1`.
3. **Red Feed-Forward**: La salida del paso anterior se pasa a través de la red feed-forward.
4. **Añadir y normalizar (después de la red feed-forward)**: Al igual que en el paso 2, la salida de la red feed-forward se añade a la entrada de esta etapa (conexión residual), seguida de una eliminación y normalización utilizando `norm2`.
5. **Salida**: El tensor procesado se devuelve como salida de la capa codificadora.   

##### <u>Resumen</u>

La clase `EncoderLayer` define una sola capa del codificador del transformer. Dispone de un mecanismo de autoatención multicabezal seguido de una red neuronal feed-forward posicional, con conexiones residuales, normalización de capas y dropout aplicados según corresponda.    

El conjunto de estos componentes permite al codificador captar relaciones complejas en los datos de entrada y transformarlas en una representación útil para las tareas posteriores. Normalmente, se apilan varias capas codificadoras para formar la parte codificadora completa de un modelo transformer.

### 4. Bloques de decodificación   

<img src="./img/decodificador_transformer.jpg" height=300>     

In [7]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

***Definición de la clase:***      

````
class DecoderLayer(nn.Module):
````   

***Inicialización:***     

````
def __init__(self, d_model, num_heads, d_ff, dropout):
    super(DecoderLayer, self).__init__()
    self.self_attn = MultiHeadAttention(d_model, num_heads)
    self.cross_attn = MultiHeadAttention(d_model, num_heads)
    self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
    self.norm1 = nn.LayerNorm(d_model)
    self.norm2 = nn.LayerNorm(d_model)
    self.norm3 = nn.LayerNorm(d_model)
    self.dropout = nn.Dropout(dropout)
````     
Parámetros:

1. `d_model`: La dimensión de la entrada.
2. `num_heads`: El número de cabezas de atención en la atención multicabeza.
3. `d_ff`: La dimensión de la capa interna en la red feed-forward.
4. `dropout`: La tasa de abandono para la regularización.   

Componentes:

1. `self.self_attn`: Mecanismo de autoatención multicabezal para la secuencia objetivo.
2. `self.cross_attn`: Mecanismo de atención multicabezal que atiende a la salida del codificador.
3. `self.feed_forward`: Red neuronal feed-forward en función de la posición.
4. `self.norm1`, `self.norm2`, `self.norm3`: Componentes de normalización de las capas.
5. `self.dropout`: Capa de abandono para la regularización.   

***Método forward:***   

`````
def forward(self, x, enc_output, src_mask, tgt_mask):
    attn_output = self.self_attn(x, x, x, tgt_mask)
    x = self.norm1(x + self.dropout(attn_output))
    attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
    x = self.norm2(x + self.dropout(attn_output))
    ff_output = self.feed_forward(x)
    x = self.norm3(x + self.dropout(ff_output))
    return x
`````   

*Entrada:*

1. `x`: La entrada a la capa decodificadora.
2. `enc_output`: La salida del codificador correspondiente (utilizada en el paso de atención cruzada).
3. `src_mask`: Máscara de origen para ignorar ciertas partes de la salida del codificador.
4. `tgt_mask`: Máscara de destino para ignorar ciertas partes de la entrada del decodificador.   


*Pasos del proceso:*

1. **Autoatención en la secuencia objetivo**: La entrada `x` se procesa a través de un mecanismo de autoatención.
2. **Añadir y normalizar (después de la autoatención)**: La salida de la autoatención se añade a la `x` original, seguida de una eliminación y normalización mediante `norm1`.
3. **Atención cruzada con la salida del codificador**: La salida normalizada del paso anterior se procesa a través de un mecanismo de atención cruzada que atiende a la salida del codificador `enc_output`.
4. **Añadir y Normalizar (después de la Atención Cruzada)**: La salida de cross-attention se añade a la entrada de esta etapa, seguida de dropout y normalización usando `norm2`.
5. **Red Feed-Forward**: La salida de la etapa anterior se pasa a través de la red feed-forward.
6. **Añadir y normalizar (después del Feed-Forward)**: La salida feed-forward se añade a la entrada de esta etapa, seguida de dropout y normalización usando `norm3`.
7. **Salida**: El tensor procesado se devuelve como salida de la capa decodificadora.   


##### <u>Resumen</u>

La clase `DecoderLayer` define una sola capa del decodificador del transformer. Consiste en un mecanismo de autoatención multicabezal, un mecanismo de atención cruzada multicabezal (que atiende a la salida del codificador), una red neuronal feed-forward en función de la posición y las correspondientes conexiones residuales, normalización de capas y capas de dropout. Esta combinación permite al descodificador generar salidas significativas basadas en las representaciones del codificador, teniendo en cuenta tanto la secuencia de destino como la secuencia de origen. Al igual que ocurre con el codificador, se suelen apilar varias capas de decodificación para formar la parte decodificadora completa de un modelo de transformer.

A continuación, en el siguiente apartado, los bloques `codificador` y `decodificador` se unen para construir el modelo transformer completo.


### 5. Transformer completo   


<img src="./img/transformer_1.jpg" height=400>

In [8]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

***Definición de la clase:***   

````
class Transformer(nn.Module):
````

***Inicialización:***   

````
def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
````    

El constructor recibe los siguientes parámetros:

1. `src_vocab_size`: Tamaño del vocabulario fuente.
2. `tgt_vocab_size`: Tamaño del vocabulario de destino.
3. `d_model`: La dimensionalidad de las incrustaciones del modelo.
4. `num_heads`: Número de cabezas de atención en el mecanismo de atención multicabeza.
5. `num_layers`: Número de capas tanto para el codificador como para el decodificador.
6. `d_ff`: Dimensionalidad de la capa interna en la red feed-forward.
7. `max_seq_length`: Longitud máxima de la secuencia para la codificación posicional.
8. `dropout`: Tasa de abandono para la regularización.    


Y define los siguientes componentes:

1. `self.encoder_embedding`: Capa de incrustación para la secuencia fuente.
2. `self.decoder_embedding`: Capa de incrustación para la secuencia de destino.
3. `self.positional_encoding`: Componente de codificación posicional.
4. `self.encoder_layers`: Una lista de capas de codificación.
5. `self.decoder_layers`: Una lista de capas decodificadoras.
6. `self.fc`: Capa final totalmente conectada (lineal) asignada al tamaño del vocabulario objetivo.
7. `self.dropout`: Capa de dropout.


***Método generador de máscara:***      

````
def generate_mask(self, src, tgt):
````   
Este método se utiliza para crear máscaras para las secuencias de origen y destino, garantizando que se ignoran los tokens de relleno(padding) y que los tokens futuros no son visibles durante el entrenamiento para la secuencia de destino.   

***Método forward:***    

````
def forward(self, src, tgt):
````    

Este método define el paso hacia delante del Transformador, tomando las secuencias de origen y destino y produciendo las predicciones de salida.

1. **Incrustación de entrada y codificación posicional**: Las secuencias de origen y destino se incrustan primero utilizando sus respectivas capas de incrustación y luego se añaden a sus codificaciones posicionales.
2. **Capas de codificación**: La secuencia fuente pasa por las capas de codificación, y la salida final del codificador representa la secuencia fuente procesada.
3. **Capas de decodificación**: La secuencia de destino y la salida del codificador pasan a través de las capas del decodificador, dando como resultado la salida del decodificador.
4. **Capa lineal final**: La salida del descodificador se asigna al tamaño del vocabulario objetivo mediante una capa totalmente conectada (lineal).   

***Salida:***    

El resultado final es un tensor que representa las predicciones del modelo para la secuencia objetivo.   


##### <u>Resumen</u>   

La clase `Transformer` reúne los diversos componentes de un modelo Transformer, incluyendo las incrustaciones, la codificación posicional, las capas codificadoras y las capas decodificadoras. Proporciona una interfaz práctica para el entrenamiento y la inferencia, encapsulando las complejidades de la atención multicabezal, las redes feed-forward y la normalización de capas.

Esta implementación sigue la arquitectura estándar de Transformer, lo que la hace adecuada para tareas de secuenciales como la traducción automática, el resumen de textos, etc. La inclusión del enmascaramiento garantiza que el modelo se adhiere a las dependencias causales dentro de las secuencias, ignorando los tokens de relleno y evitando la fuga de información de tokens futuros.

Estos pasos secuenciales permiten al modelo Transformer procesar eficazmente las secuencias de entrada y producir las correspondientes secuencias de salida.




## Entrenamiento del modelo Transformer   

### Preparación de los datos de ejemplo   

A efectos ilustrativos, en este ejemplo se utilizará un conjunto de datos ficticio. Sin embargo, en un escenario práctico, se emplearía un conjunto de datos más sustancial, y el proceso implicaría el preprocesamiento del texto junto con la creación de mapas de vocabulario tanto para el idioma de origen como para el de destino.

In [9]:
src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

# Generación de datos de ejemplo aleatorios
src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)

**Hiperparámetros:**

Estos valores definen la arquitectura y el comportamiento del modelo de transformer:

1. `src_vocab_size`, `tgt_vocab_size`: Tamaños de vocabulario para las secuencias de origen y destino, ambos fijados en 5000.
2. `d_model`: Dimensión de las incrustaciones del modelo, establecida en 512.
3. `num_heads`: Número de cabezas de atención en el mecanismo de atención multicabeza, fijado en 8.
4. `num_layers`: Número de capas tanto para el codificador como para el decodificador, establecido en 6.
5. `d_ff`: Dimensión de la capa interna en la red feed-forward, establecida en 2048.
6. `max_seq_length`: Longitud máxima de la secuencia para la codificación posicional, establecida en 100.
7. `dropout`: Tasa de abandono para la regularización, establecida en 0,1.

### Creación de una instancia del Transformer.

In [10]:
transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

La línea anterior crea una instancia de la clase `Transformer`, inicializándola con los hiperparámetros dados. La instancia tendrá la arquitectura y el comportamiento definidos por estos hiperparámetros.

***Generación de datos de muestra aleatorios:***

Las siguientes líneas generan secuencias aleatorias de origen y destino:

1. `datos_origen`: Enteros aleatorios entre 1 y `src_vocab_size`, que representan un lote de secuencias fuente con forma `(64, max_seq_length)`.
2. `tgt_data`: Números enteros aleatorios entre 1 y `tgt_vocab_size`, que representan un lote de secuencias objetivo con forma `(64, max_seq_length`)`.
Estas secuencias aleatorias pueden utilizarse como entradas para el modelo transformador, simulando un lote de datos con 64 ejemplos y secuencias de longitud 100.   


##### <u>Resumen</u>   

El fragmento de código demuestra cómo inicializar un modelo de transformador y generar secuencias aleatorias de origen y destino que pueden introducirse en el modelo. Los hiperparámetros elegidos determinan la estructura y las propiedades específicas del transformador. Esta configuración podría formar parte de un guión más amplio en el que el modelo se entrenara y evaluara en tareas reales de secuencia a secuencia, como la traducción automática o el resumen de textos.

### Entrenamiento del modelo.   

A continuación, el modelo se entrenará utilizando los datos de muestra mencionados.    

Recordar que, en un escenario real, se emplearía un conjunto de datos significativamente mayor, que normalmente se dividiría en conjuntos distintos a efectos de entrenamiento y validación.

In [11]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(100):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 8.681550979614258


: 

***Función de pérdida y optimizador:***

1. `criterion = nn.CrossEntropyLoss(ignore_index=0)`: Define la función de pérdida como pérdida de entropía cruzada. El argumento `ignore_index` se establece en 0, lo que significa que la pérdida no tendrá en cuenta los objetivos con un índice de 0 (normalmente reservado para los tokens de relleno).
2. `optimizer = optim.Adam(...)`: Define el optimizador como Adam con una tasa de aprendizaje de 0,0001 y valores beta específicos.   

***Ejecución del entrenamiento del modelo***

1. `transformer.train()`: ejecuta el entrenamiento del modelo transformer, habilitando comportamientos como el dropout que sólo se aplican durante el entrenamiento.   

***Bucle de entrenamiento:***

El fragmento de código entrena el modelo durante 100 épocas utilizando un bucle de entrenamiento típico:

1. `for epoch in range(100)`: Itera sobre 100 épocas de entrenamiento.
2. `optimizer.zero_grad()`: Borra los gradientes de la iteración anterior.
3. `output = transformer(src_data, tgt_data[:, :-1])`: Pasa los datos de origen y los datos de destino (excluyendo el último token de cada secuencia) a través del transformer. Esto es común en tareas de secuencia a secuencia en las que el destino se desplaza un token.
4. `loss = criterion(...)`: Calcula la pérdida entre las predicciones del modelo y los datos objetivo (excluyendo el primer token de cada secuencia). La pérdida se calcula transformando los datos en tensores unidimensionales y utilizando la función de pérdida de entropía cruzada.
5. `loss.backward()`: Calcula los gradientes de la pérdida con respecto a los parámetros del modelo.
6. optimizador.paso()`: Actualiza los parámetros del modelo utilizando los gradientes calculados.
7. `print(f "Epoch: {epoch+1}, Loss: {loss.item()}")`: Imprime el número de época actual y el valor de pérdida para esa época.   

##### <u>Resumen</u>   

Este fragmento de código entrena el modelo transformer en secuencias origen y destino generadas aleatoriamente durante 100 épocas. Utiliza el optimizador Adam y la función de pérdida de entropía cruzada. La pérdida se imprime para cada época, lo que le permite supervisar el progreso del entrenamiento. En un escenario real, sustituiría las secuencias de origen y destino aleatorias por datos reales de su tarea, como la traducción automática.

### Evaluación del rendimiento del modelo Transformer   

Una vez entrenado el modelo, puede evaluarse su rendimiento en un conjunto de datos de validación o de prueba.    

A continuación se ofrece un ejemplo de esto:

In [ ]:
transformer.eval()

# Generación de datos de validación aleatorios
val_src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
val_tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)

with torch.no_grad():

    val_output = transformer(val_src_data, val_tgt_data[:, :-1])
    val_loss = criterion(val_output.contiguous().view(-1, tgt_vocab_size), val_tgt_data[:, 1:].contiguous().view(-1))
    print(f"Validation Loss: {val_loss.item()}")

***Evaluación:***

1. `transformer.eval()`: Pone el modelo transformer en modo evaluación. Esto es importante porque desactiva ciertos comportamientos, como el abandono, que sólo se utilizan durante el entrenamiento.

***Generación de datos de validación aleatorios:***

1. `val_src_data`: Enteros aleatorios entre 1 y `src_vocab_size`, que representan un lote de secuencias fuente de validación con forma `(64, max_seq_length)`.
2. `val_tgt_data`: Números enteros aleatorios entre 1 y tgt_vocab_size, que representan un lote de secuencias objetivo de validación con forma `(64, max_seq_length)`.   

***Bucle de validación:***

1. `with torch.no_grad()`: Desactiva el cálculo de gradientes, ya que no necesitamos calcular gradientes durante la validación. Esto puede reducir el consumo de memoria y acelerar los cálculos.
2. `val_output = transformer(val_src_data, val_tgt_data[:, :-1])`: Pasa los datos de origen de validación y los datos de destino de validación (excluyendo el último token de cada secuencia) a través del transformador.
3. `val_loss = criterion(...)`: Calcula la pérdida entre las predicciones del modelo y los datos de destino de validación (excluyendo el primer token de cada secuencia). La pérdida se calcula transformando los datos en tensores unidimensionales y utilizando la función de pérdida de entropía cruzada definida anteriormente.
4. `print(f"Validation Loss: {val_loss.item()}")`: Imprime el valor de la pérdida de validación.

##### <u>Resumen</u>   

Este fragmento de código evalúa el modelo transformer en un conjunto de datos de validación generado aleatoriamente, calcula la pérdida de validación y la imprime.    

En el mundo real, los datos de validación aleatorios deberían sustituirse por datos de validación reales de la tarea en la que se está trabajando.    

La pérdida de validación puede dar una indicación de lo bien que el modelo está funcionando en datos no vistos, que es una medida crítica de la capacidad de generalización del modelo.

# Conclusión

En este ejemplo se ha intentado mostrar cómo construir un modelo Transformer utilizando PyTorch, una de las librerías más versátiles para el aprendizaje profundo.    

Con su capacidad de *paralelización* y de *capturar dependencias a largo plazo* en los datos, los Transformers tienen un gran potencial en varios campos, especialmente en tareas de PLN como la traducción, el resumen y el análisis de sentimiento.